In [1]:
################################
# Basic layout of the notebook #
################################
#
# 1. Load all prerequisites
# 2. Create a noise generator which outputs noise parameterized by some vars.
# 3. Create an adversary which creates adversarial images according to this noise generator.
# 4. Write a loop to display a few images (real image, regular noise, adversarial)
#
# (Unused)
# 4a. Write a loop to display a few images (real image, regular noise, adv 1, 3, 10 steps.)
# 4b. Write a function to display the averages (real image, regular noise, adv 1, 3, 10 steps.)

#########################################
# Basic layout of the problem statement #
#########################################
#
# 1. You have an image x.
# 2. You can modifiy x by adding adversarial noise to x.
# 2a. There are restrictions on the parameterization of the adversarial noise
# 2b. There are restrictions on the scale of the adversarial noise (?)


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.model_zoo as model_zoo
import torchvision.datasets as dset
import torchvision.models as models
import torchvision.transforms as trn

%matplotlib inline

/data/hendrycks/max/miniconda/envs/corruptions/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
test_transform = trn.Compose([trn.Resize(256), trn.CenterCrop(224), trn.ToTensor()])

mean = torch.FloatTensor(np.array([0.485, 0.456, 0.406]).reshape(1,3,1,1))#.cuda()
std = torch.FloatTensor(np.array([0.229, 0.224, 0.225]).reshape(1,3,1,1))#.cuda()

test_data = dset.ImageFolder('/Users/oliver/datasets/imagenette2/val/', transform=test_transform)

test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=True,
                                          num_workers=1, pin_memory=True)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/oliver/datasets/imagenette2/val/'

In [ ]:
def remove_module(state_dict):
    d = {}
    for key in state_dict:
        d[key[7:]] = state_dict[key]
    return d

net = models.resnet50()
'''
net.load_state_dict(model_zoo.load_url('https://download.pytorch.org/models/resnet50-19c8e357.pth',
                                       model_dir='/home/hendrycks/datasets/models'))
'''
session = torch.load('/Users/oliver/models/deepaugment_and_augmix.pth.tar', map_location=torch.device('cpu'))
state_dict = remove_module(session['state_dict'])
net.load_state_dict(state_dict)
#'''
net.eval()
#net.cuda()
#cudnn.benchmark = True  # fire on all cylinders

In [14]:
to_np = lambda z: z.to('cpu').detach().numpy()

In [30]:
def template_creator(vars, bsize):
    '''
    Creates some noise based on some variable. 
    Creation of noise should be batched and enable gradients 
    to flow through it.
    '''
    pass

In [25]:
class TemplateAdversary(nn.Module):
    def __init__(self, eps=0.05, scale=1, num_steps=10, step_size=0.01):
        super().__init__()
        self.eps = eps
        self.scale = scale
        self.num_steps = num_steps
        self.step_size = step_size

    def forward(self, model, bx, by, x_max):
        """
        :param model: the classifier's forward method
        :param bx: batch of images
        :param by: true labels
        :return: perturbed batch of images
        """
        bsize = bx.size(0)
        
        # create initial variables
        template_vars = 0 # TODO!!!

        # create initial images
        adv_bx = bx.detach()
        template = template_creator(template_vars, bsize) #[:,:,16:-16,16:-16]
        original_template = template.clone().detach().data

        # begin optimizing the inner loop
        opt = optim.Adam(template_vars, lr=0.01)

        for i in range(self.num_steps):
            opt.zero_grad()

            with torch.enable_grad():
                template = template_creator(template_vars, bsize) #[:,:,16:-16,16:-16]
                logits = model(((adv_bx + self.scale * template)/(x_max + self.scale) - mean)/std)
                loss = -F.cross_entropy(logits, by, reduction='sum')
            
            loss.backward(retain_graph=True)
            opt.step()
            
            # clamp variables
            for i in range(len(template_vars)):
                template_vars[i].detach()
                template_vars[i].data = template_vars[i].data.clamp(0, self.eps)
                template_vars[i].requires_grad_()

        return template, original_template

In [28]:
scale = 1
eps = 0.05
adv = TemplateAdversary(eps, scale)

In [ ]:
for i, (x, label) in enumerate(test_loader):
    if i < 3:
    
        print('\nClean Image')
        target = torch.LongTensor([label])#.cuda()

        x = x#.cuda()
        x_max, _ = torch.max(x.view(x.size(0), 3, -1), -1)
        x_max = x_max.view(-1, 3, 1, 1)

        logits = net((x - mean)/std)
        print('Loss:', to_np(F.cross_entropy(logits, target)))
        print(['Wrong Prediction', 'Right Prediction'][int(label == to_np(torch.argmax(logits, 1))[0])])
        
        plt.subplot(1,2,1) # induced for comparable size
        plt.imshow(to_np(x).squeeze().transpose((1,2,0)))
        plt.axis('off')
        plt.show()

        print('\nInitial Template')

        adv_template, original_template = adv(net, x, target, x_max)
        
        logits = net(((x + scale * original_template)/(x_max + scale) - mean)/std)
        print('Loss:', to_np(F.cross_entropy(logits, target)))
        print(['Wrong Prediction', 'Right Prediction'][int(label == to_np(torch.argmax(logits, 1))[0])])
        
        plt.subplot(1,2,1)
        plt.imshow(to_np((x + scale * original_template)/(x_max + scale)).squeeze().transpose((1,2,0)))
        plt.axis('off')
        
        plt.subplot(1,2,2)
        plt.imshow(to_np(original_template).squeeze(), cmap='gray')
        plt.axis('off')
        plt.show()
        
        print('\nAdversarial Template')
        
        logits = net(((x + scale * adv_template)/(x_max + scale) - mean)/std)
        print('Loss:', to_np(F.cross_entropy(logits, target)))
        print(['Wrong Prediction', 'Right Prediction'][int(label == to_np(torch.argmax(logits, 1))[0])])

        plt.subplot(1,2,1)
        plt.imshow(to_np((x + scale * adv_template)/(x_max + scale)).squeeze().transpose((1,2,0)))
        plt.axis('off')

        plt.subplot(1,2,2)
        plt.imshow(to_np(adv_template).squeeze(), cmap='gray')
        plt.axis('off')
        plt.show()
        
    else:
        break